In [32]:
%%bash
# Install required packages
pip install pandas
pip install sentence-transformers

#Had to add these to make it work locally
pip install matplotlib
pip install seaborn
pip install sentence_transformers
pip install nltk

In [33]:
import os
import json
import re
from typing import List, Tuple, Dict, Union
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import importlib
from collections import Counter
import random

import sys
import os

from helpers import DatasetFileLoader

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


In [66]:
import random
import pandas as pd

def to_sample_and_expected_result(data, min_input_size = 3, max_input_size=300):
    """
    This function is used to build a test set in the example format so the model can be evaluated
    Splits the provided input into sample, y_true pairs.
    Each line is split at a random index between MIN_INPUT_SIZE and MAX_INPUT_SIZE
    y_true represents the next chracter after the split. The remainder of the string is discarded

    Output:

    x: list of training samples
    y_true: list of expected outputs
    """
    x = []
    y_true = []

    for i, line in enumerate(data):
        if line != "" and type(line) == str:
            
            words = line.split()

            splittable_words = []
            
            # We only split within words since we will not need to predict whitespace
            #Since we know we will get minimum one character, of the word and will not predict whitespace, we will only split words of at least 3 characters
            for i,word in enumerate(words):
                if len(word) >= 3:
                    splittable_words.append((word, i))

            if len(splittable_words) > 0:

                #Pick a random word to split
                split_word_index = random.randint(0, len(splittable_words)-1)
                
                #Get the index and the word to split
                to_split, og_index = splittable_words[split_word_index]

                split_index = random.randint(1, len(to_split)-1) #Get a spot to split. It cannot be first or last character
                
                #split the word
                splitted = to_split[0:split_index]

                # Record the expected y value
                y = to_split[split_index]

                # Split our words to only get ones that come before the split word
                res = words[0:og_index]

                #Add back in the word we split
                res.append(splitted)

                x.append(" ".join(res))
                y_true.append(y)
        

    return x, y_true


In [68]:
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "../../data")

In [69]:
loader = DatasetFileLoader()
loader.load(f"{data_dir}/parsed_data")
loader.test_data.head()

loader.test_data = loader.test_data.drop(index=0)

In [70]:
x_test, y_test = to_sample_and_expected_result(loader.test_data[0])

In [ ]:
with open(f'{parent_dir}/../../test/input.txt', 'w') as f:
    for item in x_test:
        f.write(f"{item}\n")

In [72]:
with open(f'{parent_dir}/../../test/answer.txt', 'w') as f:
    for item in y_test:
        f.write(f"{item}\n")